In [4]:
pip install vaderSentiment

     |████████████████████████████████| 133kB 7.0MB/s 


In [5]:
import pandas as pd
import numpy as np
from textblob import TextBlob
import re
import nltk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [6]:
from google.colab import files
uploaded = files.upload()

Saving Company 1.csv to Company 1 (1).csv
Saving Company 2.csv to Company 2 (1).csv


In [7]:
df1 = pd.read_csv('Company 1.csv')
df2 = pd.read_csv('Company 2.csv')

In [8]:
df1.head(3)
df2.head(3)

,Date,Open,High,Low,Last,Close,Total Trade Quantity,Turnover (Lacs)
0,2010-01-04,1613.00,1629.10,1602.00,1629.0,1625.65,9365.0,151.74
1,2010-01-05,1639.95,1639.95,1611.05,1620.0,1616.80,38148.0,622.58
2,2010-01-06,1618.00,1644.00,1617.00,1639.0,1638.50,36519.0,595.09


In [9]:
merge = df1.merge(df2, how='inner', on='Date', left_index = True)
merge.head(3)

,Date,Open_x,High_x,Low_x,Last_x,Close_x,Total Trade Quantity_x,Turnover (Lacs)_x,Open_y,High_y,Low_y,Last_y,Close_y,Total Trade Quantity_y,Turnover (Lacs)_y
0,2010-01-04,1121.0,1151.00,1121.00,1134.0,1135.6,101651.0,1157.18,1613.00,1629.10,1602.00,1629.0,1625.65,9365.0,151.74
1,2010-01-05,1146.8,1149.00,1128.75,1135.0,1134.6,59504.0,676.47,1639.95,1639.95,1611.05,1620.0,1616.80,38148.0,622.58
2,2010-01-06,1140.0,1164.25,1130.05,1137.0,1139.6,128908.0,1482.84,1618.00,1644.00,1617.00,1639.0,1638.50,36519.0,595.09


In [14]:
headlines = [] 
for row in range(0,len(merge.index)):
    headlines.append(' '.join(str(x) for x in merge.iloc[row,2:27]))

In [20]:
 clean_headlines = []
 for i in range(0, len(headlines)):
   clean_headlines.append(re.sub("b[(')]+", '', headlines[i] ))
   clean_headlines[i] = re.sub('b[(")]+', '', clean_headlines[i] )
   clean_headlines[i] = re.sub("\'", '', clean_headlines[i]) 

In [21]:
merge['Combined_News'] = clean_headlines

In [22]:
#function to get the subjectivity
def getSubjectivity(text):
  return TextBlob(text).sentiment.subjectivity
# function to get the polarity
def getPolarity(text):
  return  TextBlob(text).sentiment.polarity

In [23]:
merge['Subjectivity'] =merge['Combined_News'].apply(getSubjectivity)
merge['Polarity'] =merge['Combined_News'].apply(getPolarity)

In [26]:
#function to get the sentiment scores (using Sentiment Intensity Analyzer)
def getSIA(text):
  sia = SentimentIntensityAnalyzer()  
  sentiment = sia.polarity_scores(text)
  return sentiment

In [28]:
compound = []
neg = []
neu = []
pos = []
SIA = 0
for i in range(0, len(merge['Combined_News'])):
  SIA = getSIA(merge['Combined_News'][i])
  compound.append(SIA['compound'])
  neg.append(SIA['neg'])
  neu.append(SIA['neu'])
  pos.append(SIA['pos'])

In [29]:
merge['Compound'] =compound
merge['Negative'] =neg
merge['Neutral'] =neu
merge['Positive'] = pos

In [39]:
keep_columns=['Subjectivity','Polarity','Compound','Negative','Neutral','Positive']
df= merge[keep_columns]
df

,Subjectivity,Polarity,Compound,Negative,Neutral,Positive
0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...
1734,0.0,0.0,0.0,0.0,1.0,0.0
1735,0.0,0.0,0.0,0.0,1.0,0.0
1736,0.0,0.0,0.0,0.0,1.0,0.0
1737,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
X = df
X = np.array(X.drop(['Label'],1))#Create the target data set
y = np.array(df['Label'])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
model = LinearDiscriminantAnalysis().fit(x_train, y_train)

In [ ]:
predictions = model.predict(x_test)
predictions

In [ ]:
print( classification_report(y_test, predictions) )